In [5]:
# This should be built on a fully commissioned ESP32
# But there's problems with getting both i2c devices 
# working from an ESP32

# So we have an MQTT version which you can get the data using
# mosquitto_sub -h 192.168.0.159 -v -t "walkingpole/#"

In [61]:
%serialconnect

Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.


In [99]:
# the location of the i2c pins on the firebeetle
%sendtofile i2cmodule.py
import machine
#i2c = machine.I2C(scl=machine.Pin(22), sda=machine.Pin(21))
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5), freq=100000)  #D2,D1 NodeMCU


Sent 3 lines (163 bytes) to i2cmodule.py.


In [161]:
%sendtofile --source ../essential_sensor_code/VL53L0X_funcs.py
%sendtofile --source ../essential_sensor_code/BNO055_funcs.py

Sent 46 lines (1653 bytes) to BNO055_funcs.py.


In [63]:
%sendtofile main.py

#ssid, ssidpassword = b'DoESLiverpool', b'decafbad00'
#mqttbroker = "192.168.0.159"   # mqtt.local
ssid = None

from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
from BNO055_funcs import BNO055init, BNO055calibstat, BNO055quat

from machine import Pin
import time, network, math
from umqtt.robust import MQTTClient

pled = Pin(2, Pin.OUT)

if ssid:
    staif = network.WLAN(network.STA_IF)
    staif.active(True)
    staif.connect(ssid, ssidpassword)
    while not staif.isconnected():
        pled.value(1-pled.value())
        time.sleep(0.2)
else:
    staif = network.WLAN(network.AP_IF)
    staif.active(True)
    
VL53L0Xinit()
BNO055init()

mqttbroker = None
mqttclient = None

t0 = time.ticks_ms()
i = 0
while True:
    i += 1
    pled.value(i%2)
    time.sleep_ms(5)
    t1 = time.ticks_ms()
    time.sleep_ms(max(5, 50-(t1-t0)))
    t1 = time.ticks_ms()
    pled.value(1)
    
    if mqttbroker is None:
        mqttbroker = "192.168.4.%d"%((i%4)+2)
        print("trying mqttbroker", mqttbroker)
    if mqttclient is None:
        mqttclient = MQTTClient(b"walkingpole", mqttbroker)
        try:
            mqttclient.connect()
            mqttclient.publish(b"walkingpole/ip", staif.ifconfig()[0].encode())
        except OSError as e:
            print(e)
            if ssid is None:
                mqttbroker = None
            mqttclient = None
            continue
    
    q0, q1, q2, q3 = BNO055quat()
    d = VL53L0Xdist()
    
    riqsq = q0*q0 + q1*q1 + q2*q2 + q3*q3 
    iqsq = 1/max(1,riqsq)
    r31 = 2*(q1*q3 - q0*q2)*iqsq
    r32 = 2*(q0*q1 + q2*q3)*iqsq
    r33 = 1 - 2*(q1**2 + q2**2)*iqsq
    sidetilt = math.degrees(-math.atan2(r33, r32))
    foretilt = math.degrees(-math.atan2(r31, r32))

    msg = b"{} {} {} {}".format(t1-t0, int(d*4), int(foretilt), int(sidetilt))
    print(msg)
    .publish(b"walkingpole/dtilt", msg)
    t0 = t1



Sent 72 lines (1872 bytes) to main.py.


In [59]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft#6 ets_task(40100130, 3, 3fff837c, 4)\r\nWebREPL is not configured, run \'import webrepl_setup\'\r\n'
not command mode
BNO055 sensor SW_REV_ID: 0x11.0x3
trying mqttbroker 192.168.4.43
51 69.25 96.3353 -91.6805
357 69.25 96.3353 -91.6805
51 0 96.3353 -91.6805
51 69.25 96.3353 -91.6805
50 69.0 96.3353 -91.6805
50 68.25 96.3353 -91.6805
50 0 96.3353 -91.6805
51 69.5 96.3353 -91.6805
51 69.0 96.3353 -91.6805
50 69.75 96.3353 -91.6805
51 69.5 96.3353 -91.6805
50 69.0 96.3353 -91.6805
50 69.5 96.3353 -91.6805
50 69.5 96.3353 -91.6805
50 69.0 96.3353 -91.6805
50 68.75 96.3353 -91.6805
50 69.75 96.3353 -91.6805
50 69.5 96.3353 -91.6805
50 68.75 96.3353 -91.6805
50 69.0 96.3353 -91.6805
50 68.75 96.3353 -91.6805
50 70.25 96.3353 -91.6805
50 69.25 96.3353 -91.6805
50 69.5 96.3353 -91.6805
50 69.5 96.3353 -91.6805
51 69.25 96

50 61.5 85.3568 -88.7442
51 61.25 85.3552 -88.7437
50 62.0 85.3606 -88.7452
51 61.5 85.3606 -88.7452
50 61.0 85.3606 -88.7452
51 61.25 85.3695 -88.7477
51 61.0 85.3625 -88.7458
51 60.5 85.3568 -88.7442
50 61.5 85.3517 -88.7428
50 61.75 85.3641 -88.7462
50 61.5 85.3501 -88.7424
50 0 85.3533 -88.7432
50 60.75 85.3606 -88.7453
51 61.5 85.3568 -88.7442
50 61.5 85.3658 -88.7466
50 61.5 85.3712 -88.7481
51 60.75 85.3501 -88.7424
51 61.75 85.3677 -88.7471
51 60.5 85.3517 -88.7428
51 61.0 85.3568 -88.7442
51 61.75 85.3712 -88.7481
50 61.5 85.3693 -88.7476
51 61.5 85.3693 -88.7476
51 61.5 85.3498 -88.7423
50 61.25 85.3498 -88.7423
50 61.25 85.3677 -88.7471
50 0 85.3517 -88.7428
51 0 85.3517 -88.7428
50 62.25 85.3587 -88.7447
51 61.25 85.3587 -88.7447
50 60.75 85.3587 -88.7447
51 60.75 85.3587 -88.7447
50 61.5 85.3587 -88.7447
50 0 85.3587 -88.7447
50 0 85.3587 -88.7447
50 60.5 85.3587 -88.7447
51 0 85.3587 -88.7447
50 59.5 85.3587 -88.7447
51 59.5 85.3587 -88.7447
51 0 85.3587 -88.7447
50 0 85.

51 94.5 85.3587 -88.7447
51 93.25 85.3587 -88.7447
51 95.0 85.3587 -88.7447
51 94.75 85.3587 -88.7447
50 94.5 85.3587 -88.7447
50 94.75 85.3587 -88.7447
50 0 85.3587 -88.7447
50 94.25 85.3587 -88.7447
51 94.5 85.3587 -88.7447
50 93.5 85.3587 -88.7447
51 95.0 85.3587 -88.7447
51 93.5 85.3587 -88.7447
50 94.75 85.3587 -88.7447
50 94.5 85.3587 -88.7447
50 94.5 85.3587 -88.7447
50 94.5 85.3587 -88.7447
50 94.75 85.3587 -88.7447
50 94.25 85.3587 -88.7447
50 94.0 85.3587 -88.7447
50 94.75 85.3587 -88.7447
50 95.0 85.3587 -88.7447
51 94.75 85.3587 -88.7447
51 95.0 85.3587 -88.7447
51 0 85.3587 -88.7447
51 95.0 85.3587 -88.7447
50 93.25 85.3587 -88.7447
51 93.25 85.3587 -88.7447
52 94.75 85.3587 -88.7447
50 95.75 85.3587 -88.7447
51 94.75 85.3587 -88.7447
50 94.5 85.3587 -88.7447
51 93.25 85.3587 -88.7447
51 94.0 85.3587 -88.7447
51 94.25 85.3587 -88.7447
50 94.0 85.3587 -88.7447
50 93.75 85.3587 -88.7447
50 94.25 85.3587 -88.7447
51 94.25 85.3587 -88.7447
50 94.75 85.3587 -88.7447
51 95.75 85

In [55]:
1

[leftinbuffer] ['[Errno 103] ECONNABORTED']
[leftinbuffer] ['trying mqttbroker 192.168.4.4']
[leftinbuffer] ['[Errno 103] ECONNABORTED']
[leftinbuffer] ['trying mqttbroker 192.168.4.5']
[leftinbuffer] ['[Errno 103] ECONNABORTED']
[leftinbuffer] ['trying mqttbroker 192.168.4.2']
[leftinbuffer] ['[Errno 103] ECONNABORTED']
[leftinbuffer] ['trying mqttbroker 192.168.4.3']

[missing-OK][Errno 103] ECONNABORTED

[missing-OK]trying mqttbroker 192.168.4.4
..
[missing-OK][Errno 103] ECONNABORTED

[missing-OK]trying mqttbroker 192.168.4.5
.
[missing-OK][Errno 103] ECONNABORTED

[missing-OK]trying mqttbroker 192.168.4.2
.
[missing-OK][Errno 103] ECONNABORTED

[missing-OK]trying mqttbroker 192.168.4.3
.
[missing-OK][Errno 103] ECONNABORTED

[missing-OK]trying mqttbroker 192.168.4.4
.
[missing-OK][Errno 103] ECONNABORTED

[missing-OK]trying mqttbroker 192.168.4.5
..
[missing-OK][Errno 103] ECONNABORTED

[missing-OK]trying mqttbroker 192.168.4.2
.
[missing-OK][Errno 103] ECONNABORTED

[missing-OK]t

In [2]:
# Working out code below
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [3]:
from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist

In [4]:
import time
VL53L0Xinit()
for i in range(10):
    print(VL53L0Xdist())
    time.sleep(1)

35.75
36.25
37.75
43.5
38.5
.33.25
26.5
25.75
0
0


In [5]:
from i2cmodule import i2c
print(i2c.scan())

[40, 41]


In [6]:
k = i2c.readfrom_mem(0x28, 0x00, 6)
print("BNO055 sensor SW_REV_ID: %s.%s" %(hex(k[4]), hex(k[5])))


BNO055 sensor SW_REV_ID: 0x11.0x3


In [18]:
from BNO055_funcs import BNO055init, BNO055calibstat, BNO055quat, BNO055pitchrollorient
BNO055init()


BNO055 sensor SW_REV_ID: 0x11.0x3


In [ ]:
# pull rows from rotation matrix
# https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
row 1
    x = 1-2*(q2**2 + q3**2)  
    y = 2*(q1*q2 - q0*q3)
    z = 2*(q0*q2 + q1*q3)
row 2:
    x = 2*(q1*q2 + q0*q3)
    y = 1 - 2*(q1**2 + q3**2)
    z = 2*(q2*q3 - q0*q1)

    2*(q1*q3 - q0*q2)
2*(q0*q1 + q2*q3)
1 - 2*(q1**2 + q2**2)

    
    

In [46]:
print(math.atan2(0.1,1))

0.0996686


In [48]:
#BNO055init()
for i in range(10):
    q0, q1, q2, q3 = BNO055quat()
    riqsq = q0*q0 + q1*q1 + q2*q2 + q3*q3 
    iqsq = 1/max(1,riqsq)
    
    # row 3
    r31 = 2*(q1*q3 - q0*q2)*iqsq
    r32 = 2*(q0*q1 + q2*q3)*iqsq
    r33 = 1 - 2*(q1**2 + q2**2)*iqsq
    
    # r32 is down vector
    sidetilt = math.degrees(-math.atan2(r33, r32))
    foretilt = math.degrees(-math.atan2(r31, r32))
    print(foretilt, sidetilt)
    time.sleep(2)
    

90.7416 -90.1934
-4.09704 -9.60139
.-0.174708 16.2851
-1.70639 9.95241
2.67665 -16.5084
.89.798 -89.9377
90.8165 -90.2136
90.7892 -90.2064
90.7892 -90.2064
.90.7927 -90.2072


In [33]:
import math
def gg():
    w, x, y, z = BNO055quat()
    ysqr = y * y

    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + ysqr)
    X = math.degrees(math.atan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.degrees(math.asin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (ysqr + z * z)
    Z = math.degrees(math.atan2(t3, t4))
    print(X, Y, Z) # roll, pitch, yaw


In [34]:
for i in range(20):
    gg()
    time.sleep(2)

175.262 90.0 -135.375
175.211 90.0 -135.379
115.575 90.0 -115.593
.122.116 90.0 -136.48
148.77 90.0 -106.005
91.5964 -90.0 -130.071
.91.2361 90.0 -139.063
129.99 90.0 -129.894
172.098 90.0 -137.776
.166.897 90.0 -138.778
-93.193 90.0 -135.418
173.074 90.0 -139.47
.-97.5004 90.0 -135.425
105.726 90.0 -133.448
106.352 90.0 -133.274
.112.034 90.0 -137.758
97.6242 90.0 -135.096
.-137.872 90.0 -138.3
-136.725 90.0 -138.293
-136.725 90.0 -138.293
.

In [25]:
    q0, q1, q2, q3 = ustruct.unpack("<hhhh", i2c.readfrom_mem(0x28, 0x20, 8))
    riqsq = q0*q0 + q1*q1 + q2*q2 + q3*q3 
    iqsq = 1/riqsq 
    
    r02 = q0*q2*2 * iqsq
    r13 = q1*q3*2 * iqsq
    sinpitch = r13 - r02

    r01 = q0*q1*2 * iqsq
    r23 = q2*q3*2 * iqsq 
    sinroll = r23 + r01 
     
    r00 = q0*q0*2 * iqsq
    r11 = q1*q1*2 * iqsq
    r03 = q0*q3*2 * iqsq
    r12 = q1*q2*2 * iqsq
    a00=r00 - 1 + r11   
    a01=r12 + r03  
    rads = math.atan2(a00, -a01) 
    northorient = 180 - math.degrees(rads) 
    return math.degrees(math.asin(sinpitch)), math.degrees(math.asin(sinroll)), northorient


Traceback (most recent call last):
  File "<stdin>", line 1
IndentationError: unexpected indent


In [274]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [205]:
x = [1,2]
def f(a,b,c):
    print(c)
f(9, *x)

2


In [223]:
help(network)

object <module 'network'> is of type module
  __name__ -- network
  WLAN -- <function>
  phy_mode -- <function>
  STA_IF -- 0
  AP_IF -- 1
  STAT_IDLE -- 0
  STAT_CONNECTING -- 1
  STAT_WRONG_PASSWORD -- 2
  STAT_NO_AP_FOUND -- 3
  STAT_CONNECT_FAIL -- 4
  STAT_GOT_IP -- 5
  MODE_11B -- 1
  MODE_11G -- 2
  MODE_11N -- 3
  AUTH_OPEN -- 0
  AUTH_WEP -- 1
  AUTH_WPA_PSK -- 2
  AUTH_WPA2_PSK -- 3
  AUTH_WPA_WPA2_PSK -- 4


In [224]:
import network
staif = network.WLAN(network.AP_IF)
staif.active(True)


In [242]:
help(staif)

object <WLAN> is of type WLAN
  active -- <function>
  connect -- <function>
  disconnect -- <function>
  status -- <function>
  scan -- <function>
  isconnected -- <function>
  config -- <function>
  ifconfig -- <function>


In [245]:
print(staif.config("essid"))
print(staif.status("stations"))


MicroPython-ac74e4


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
TypeError: function takes 1 positional arguments but 2 were given


In [263]:
client = MQTTClient(b"walkingpole", '192.168.4.4')
client.connect()
client.publish(b"walkingpole/ip", staif.ifconfig()[0].encode())

In [265]:
client.publish(b"walkingpole/duung", b"dong")


In [268]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f237e71b198, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
